In [2]:
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
import torch
import os
import sys
#sys.path.append("/home/prasad/pytorch-unet/")
from pre_process import Preprocessed_additional_data
from pre_process import Preprocessed_1day_data
from pre_process import read1day_data

In [1]:
from tensorflow import keras
model = keras.models.load_model("/home/prasad/git/conv_lstm/visconv_h984_w3_1_loss_345.h5" )
#tuned_27_9_conv_h984_w3_9_additionald_2_loss_tr1224_v651.h5 602.13855 w=3 no ad
#conv_h984_w3_9_additionald_2_loss_tr1224_v651.h5            644.6068  w=2 no ad2

#conv_h984_norm_w5_8_loss_ 0_1398t_0_0828.h5    w=5  no8  886.1269     
#conv_h984_w6_7_loss_1588t_820v.h5              w=6  no7  828.2111
#conv_h984_w5_6_loss_1585t_818v.h5              w=5  no6  888.12665
#conv_h984_w4_5_loss_1274.h5                    w=4  no5  807.4869
#conv_h984_w3_4_loss_1443.h5                    w=3  no4  726.41455


In [3]:
dir_path='/home/prasad/pytorch-unet/INSAT3D_VIS_India/'


#X,y,verify=read1day_data(dir_path=dir_path,inp_seq_len=3,pred_frame=1,normalized_output='no')
X,y,maxx,minn,verify=Preprocessed_1day_data(dir_path,normalization_type='scaling',inp_seq_len=3,pred_frame=1,normalized_output='no')

In [8]:
def mse_on1day(model,X,y,mean=0,std_dev=0,normalized_output='no'):   #
    MSE=[]
    i=0
    for imgs in X:
        inp=imgs
        inp= np.reshape(inp,(1,inp.shape[0],inp.shape[1],inp.shape[2],inp.shape[3]))
        pred= model.predict(inp)
        pred= np.reshape(pred, (pred.shape[0],pred.shape[1],pred.shape[2]))
        output=pred[0]
        if normalized_output=="yes":
            output = output*std_dev + mean  
            
        ground_truth=y[i]
        ground_truth=np.reshape(ground_truth,(ground_truth.shape[0],ground_truth.shape[1]))
        mse = ((output-ground_truth)**2).mean()
        #print(inp.shape,pred.shape,output.shape,ground_truth.shape,mse)
        MSE.append(mse)

        i=i+1
    return MSE

In [9]:
mse=mse_on1day(model,X,y)

In [10]:
mse 

[449.29514,
 621.6222,
 836.2679,
 1048.8015,
 1263.7893,
 1398.4438,
 1537.8429,
 1678.0725,
 1749.5332,
 1754.2477,
 1641.3884,
 1488.946,
 1311.8076,
 1205.5815,
 987.25494,
 718.0847,
 500.7884,
 334.7693,
 215.69487,
 126.03116,
 75.85275,
 32.869514,
 12.674273,
 5.0171967,
 3.0881302,
 2.545144,
 2.6136253,
 3.1254084,
 4.810268,
 8.584832,
 21.18128,
 2.2066512,
 3.0286674,
 2.0715733,
 5.0379763,
 8.672766]

In [11]:
def bmse_on1day(X,y,mean=0,std_dev=0):   #

    MSE=[]
    i=0
    for imgs in X:
        inp=imgs
        inp=imgs[-1]
        inp=np.reshape(inp,(inp.shape[0],inp.shape[1]))
     
            
        ground_truth=y[i]
        ground_truth=np.reshape(ground_truth,(ground_truth.shape[0],ground_truth.shape[1]))
        mse = ((inp-ground_truth)**2).mean()
        #print(inp.shape,pred.shape,output.shape,ground_truth.shape,mse)
        MSE.append(mse)

        i=i+1
    return MSE

In [12]:
bmse_on1day(X,y)

[468.8575,
 598.53204,
 790.58356,
 969.23553,
 1098.8923,
 1311.247,
 1405.9524,
 1447.9839,
 1611.856,
 1686.538,
 1718.8212,
 1757.1769,
 1616.449,
 1850.8038,
 1578.8099,
 1515.6066,
 1246.5443,
 985.46875,
 769.22754,
 469.41855,
 277.7273,
 125.54673,
 45.545094,
 15.316513,
 3.252323,
 2.8435876,
 2.797083,
 2.7995896,
 3.8849242,
 7.182833,
 18.974054,
 3.0296211,
 2.6751401,
 2.6952772,
 2.9315066,
 8.921081]

In [ ]:
#def bmse()


latest = np.array( Image.open( os.path.join(dir_path,inp_files[-1]) ) )
ground_truth = np.array( Image.open( os.path.join( dir_path, target_file ) ) )

bmse = ((latest-ground_truth)**2).mean()
print( " Baseline MSE -  ", bmse )

In [ ]:
"""
model= 'norm'
inp = []
mean, std=661.4041, 110.79243  #training mean std

for files in inp_files:
    inp.append(  np.array( Image.open( os.path.join(dir_path,files ) )) )

inp = np.array(inp)
inp = (inp-mean)/std
inp = np.expand_dims(inp,axis=0)
inp =np.reshape(inp, (inp.shape[0], inp.shape[1], inp.shape[2], inp.shape[3],1))
inp.shape

pred= Model.predict(inp)
inp.shape,pred.shape

pred= np.reshape(pred, (pred.shape[0],pred.shape[1],pred.shape[2]))
output=pred[0]
output = output*std + mean

mse = ((output-ground_truth)**2).mean()
print( "Connvlstm MSE - ",mse )
Image.fromarray(output).save("unet_pred.tif")"""


In [ ]:
"""
0mn,mx = 519,948

bmse = ((latest-ground_truth)**2).mean()
scaled_latest = (latest - mn)/(mx-mn)
scaled_ground_truth = ( ground_truth - mn )/( mx-mn )
sbmse = (  ((scaled_latest - scaled_ground_truth)**2).mean()  )

scaled_output = (output - mn)/(mx-mn)
s_unet_mse = ( ((scaled_output - scaled_ground_truth)**2).mean() )"""